# Calculating metrics for explorer runs

Using [nervaluate](https://github.com/MantisAI/nervaluate).

In [ ]:
!{sys.executable} -m pip install nervaluate

In [1]:
import ast
import glob
import os
import hashlib
import itertools
from collections import defaultdict

import argilla as rg
import numpy as np
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import spacy
import math
from functools import partial
from tqdm.auto import tqdm

In [2]:
# Create mapping from old to new IDs
mapping_path = "/Users/kalyan/Documents/CPR/unfccc-global-stocktake-documents/notebooks/old-to-new-dataset-mapping.csv"
mapping_df = pd.read_csv(mapping_path)

ids_mapping = {
    row["document_id_old"]: row["document_id_new"] for _, row in mapping_df.iterrows()
}

In [3]:
load_dotenv(find_dotenv())

nlp = spacy.blank("en")

DATASET_NAME = "explorer-quality-testing"

rg.init(workspace="gst", api_key=os.environ["ARGILLA_API_KEY"])
rg_dataset = rg.load(DATASET_NAME, query="status:Validated")

# List of subdirectories to include
concepts = [
    "financial-flows",
    "deforestation",
    "vulnerable-groups",
    "equity-and-just-transition",
    "barriers-and-challenges",
    "good-practice-and-opportunities",
]

type_mapping = {
    "Good Practices and Opportunities": "Good Practice And Opportunities",
    "Barriers and Challenges": "Barriers And Challenges",
    "Equity And Justice": "Equity And Just Transition",
}


def map_type(old_type: str) -> str:
    """Map type used in Argilla to any renamed types since the Argilla load"""
    if old_type not in type_mapping:
        return old_type

    return type_mapping[old_type]


def map_record_id(record):
    """Modify a TokenClassificationRecord in place to use new IDs"""

    new_id = ids_mapping.get(record.metadata["document_id"])

    if new_id is None:
        # print(f"could not find ID {record.metadata['document_id']} in mapping")

        return None

    record.metadata["document_id"] = new_id

    return record


mapped_documents = 0

for idx, document in enumerate(rg_dataset):
    if document.annotation:
        document.annotation = [
            (map_type(item[0]), item[1], item[2]) for item in document.annotation
        ]

    # # FIXME: we exclude annotations from Joe here as his labelling style was a bit different
    # if document.annotation_agent == "joe":
    #     document.metadata["document_id"] = "EXCLUDED"

    if map_record_id(document) is None:
        continue

    else:
        document = map_record_id(document)
        mapped_documents += 1


print(f"{mapped_documents}/{len(rg_dataset)} documents successfully mapped ids")

759/785 documents successfully mapped ids


## Load CPR dataset and add span annotations to token classification records

In [4]:
import sys

sys.path.append("../..")

from pathlib import Path

from src.opensearch.index_data import get_dataset_and_filter_values

In [5]:
cpr_dataset, _ = get_dataset_and_filter_values(
    Path(os.environ["DOCS_DIR_GST"]),
    Path(
        "/Users/kalyan/Documents/CPR/unfccc-global-stocktake-documents/CPR_UNFCCC_MASTER.csv"
    ),
    Path("../../concepts").absolute(),
    limit=None,
)

  0%|          | 0/1697 [00:00<?, ?it/s]

  0%|          | 0/1679 [00:00<?, ?it/s]

0docs [00:00, ?docs/s]

In [6]:
# create hash map between IDs and spans

SPAN_TYPES = {
    "Financial Flows",
    "Deforestation",
    "Vulnerable Groups",
    "Equity And Just Transition",
    "Barriers And Challenges",
    "Good Practice And Opportunities",
}

doc_id_text_hash_span_map = dict()

for doc in tqdm(cpr_dataset.documents):
    if doc.text_blocks is None:
        continue

    for block in doc.text_blocks:
        # We have to recreate the block text hash here as hashes aren't stored in argilla
        # and we modify the text on argilla data load. This is replicating the transformation
        # that happens in Argilla
        text = block.to_string().replace("\n", " ").replace("  ", " ")
        text_hash = hashlib.md5(text.encode("utf-8")).hexdigest()

        for span in block.spans:
            if span.type.split("–")[0].strip() not in SPAN_TYPES:
                continue

            span.type = span.type.split("–")[0].strip()

            doc_span_id = f"{span.document_id}_{text_hash}"

            if doc_span_id in doc_id_text_hash_span_map:
                doc_id_text_hash_span_map[doc_span_id].append(span)

            else:
                doc_id_text_hash_span_map[doc_span_id] = [span]

  0%|          | 0/1679 [00:00<?, ?it/s]

In [12]:
# add predictions to each Argilla record

# Maps span type to records which predict only for that span type to deal with the fact that Argilla can't handle overlaps
prediction_records = defaultdict(list)

argilla_blocks_found_in_cpr_dataset = 0

for record in tqdm(rg_dataset):
    text_hash = hashlib.md5(record.text.encode("utf-8")).hexdigest()
    doc_span_id = f"{record.metadata['document_id']}_{text_hash}"

    if doc_span_id in doc_id_text_hash_span_map:
        argilla_blocks_found_in_cpr_dataset += 1

        predictions = [
            (span.type, span.start_idx, span.end_idx)
            for span in doc_id_text_hash_span_map[doc_span_id]
        ]
        predictions = sorted(predictions, key=lambda x: x[0])

        # add all predictions to argilla record
        record.prediction = predictions

        # create temp argilla records per span type
        for span_type, span_predictions in itertools.groupby(
            predictions, lambda x: x[0]
        ):
            record_copy = record.copy()
            record_copy.prediction = list(span_predictions)
            prediction_records[span_type].append(record_copy)

print(f"Argilla blocks found in CPR dataset: {argilla_blocks_found_in_cpr_dataset}")
support = {k: len(v) for k, v in prediction_records.items()}
support

  0%|          | 0/785 [00:00<?, ?it/s]

Argilla blocks found in CPR dataset: 154


{'Good Practice And Opportunities': 38,
 'Barriers And Challenges': 39,
 'Deforestation': 39,
 'Equity And Just Transition': 21,
 'Financial Flows': 43,
 'Vulnerable Groups': 20}

## Use nervaluate for performance stats measurement

In [8]:
from nervaluate import Evaluator

In [9]:
def argilla_to_nervaluate(record):
    return [{"label": item[0], "start": item[1], "end": item[2]} for item in record]


ground_truth = [argilla_to_nervaluate(record.annotation) for record in rg_dataset]
predictions = [argilla_to_nervaluate(record.prediction or []) for record in rg_dataset]

In [10]:
results, results_per_tag = Evaluator(ground_truth, predictions, SPAN_TYPES).evaluate()

In [15]:
# We only want ent_type (any overlap between ground truth and prediction for an entity) or exact (exact boundaries and matching type) for each span type

ner_results_ent_type = dict()
ner_results_strict = dict()

for span_type, results in results_per_tag.items():
    ner_results_ent_type[span_type] = results["ent_type"]
    ner_results_strict[span_type] = results["strict"]

print("Results - any overlap between prediction and actual")
results_df_ent_type = pd.DataFrame(ner_results_ent_type).T
results_df_ent_type["support"] = pd.Series(support)
display(results_df_ent_type.round(2))

print()
print("Results, strict - exact boundary match between prediction and actual")
results_df_strict = pd.DataFrame(ner_results_strict).T
results_df_strict["support"] = pd.Series(support)
display(results_df_strict.round(2))

Results - any overlap between prediction and actual


,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1,support
Financial Flows,40.0,1.0,0.0,46.0,102.0,87.0,143.0,0.28,0.46,0.35,43
Vulnerable Groups,19.0,0.0,0.0,28.0,5.0,47.0,24.0,0.79,0.40,0.54,20
Equity And Just Transition,29.0,10.0,0.0,46.0,6.0,85.0,45.0,0.64,0.34,0.45,21
Barriers And Challenges,27.0,3.0,0.0,25.0,20.0,55.0,50.0,0.54,0.49,0.51,39
Good Practice And Opportunities,30.0,0.0,0.0,25.0,20.0,55.0,50.0,0.60,0.55,0.57,38
Deforestation,63.0,1.0,0.0,40.0,36.0,104.0,100.0,0.63,0.61,0.62,39



Results, strict - exact boundary match between prediction and actual


,correct,incorrect,partial,missed,spurious,possible,actual,precision,recall,f1,support
Financial Flows,8.0,33.0,0.0,46.0,102.0,87.0,143.0,0.06,0.09,0.07,43
Vulnerable Groups,9.0,10.0,0.0,28.0,5.0,47.0,24.0,0.38,0.19,0.25,20
Equity And Just Transition,14.0,25.0,0.0,46.0,6.0,85.0,45.0,0.31,0.16,0.22,21
Barriers And Challenges,12.0,18.0,0.0,25.0,20.0,55.0,50.0,0.24,0.22,0.23,39
Good Practice And Opportunities,20.0,10.0,0.0,25.0,20.0,55.0,50.0,0.40,0.36,0.38,38
Deforestation,26.0,38.0,0.0,40.0,36.0,104.0,100.0,0.26,0.25,0.25,39


### Print results to tables for methodology

`ent_type`: precision, recall, f1, support

In [24]:
for concept, row in pd.DataFrame(results_df_ent_type).sort_index().iterrows():
    concept_table = row[["precision", "recall", "f1", "support"]].rename(
        lambda i: i.title()
    )
    concept_table[["Precision", "Recall", "F1"]] = concept_table[
        ["Precision", "Recall", "F1"]
    ].round(2)

    print(concept)
    print(pd.DataFrame(concept_table).T.to_markdown(index=False))

Barriers And Challenges
|   Precision |   Recall |   F1 |   Support |
|------------:|---------:|-----:|----------:|
|        0.54 |     0.49 | 0.51 |        39 |
Deforestation
|   Precision |   Recall |   F1 |   Support |
|------------:|---------:|-----:|----------:|
|        0.63 |     0.61 | 0.62 |        39 |
Equity And Just Transition
|   Precision |   Recall |   F1 |   Support |
|------------:|---------:|-----:|----------:|
|        0.64 |     0.34 | 0.45 |        21 |
Financial Flows
|   Precision |   Recall |   F1 |   Support |
|------------:|---------:|-----:|----------:|
|        0.28 |     0.46 | 0.35 |        43 |
Good Practice And Opportunities
|   Precision |   Recall |   F1 |   Support |
|------------:|---------:|-----:|----------:|
|         0.6 |     0.55 | 0.57 |        38 |
Vulnerable Groups
|   Precision |   Recall |   F1 |   Support |
|------------:|---------:|-----:|----------:|
|        0.79 |      0.4 | 0.54 |        20 |
